<a href="https://colab.research.google.com/github/SciEcon/bitcoin_golden_litecoin_silver/blob/main/UTXO_Automatic_Updater.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instructions:

Before you run the program, please

1. Add shortcut for the shared **UTXO folder** as a copy to your own google drive 
2. Make a copy of the Colab Notebook shared by your collaborator who made the last updates
3. Change variables **currency** and **"currency_brief"** to currency of your choice. 

> The choice set = {'litecoin', 'dogecoin', 'dash', 'zcash', 'bitcoin_cash'}

> The choice_brief set = {'ltc', 'dgc', 'dash', 'zec', 'bch'}

4. Change the variable **last_update** and (update) **end** to the date of your choice

After you run the program, please 

1.   Update the lastest update date below.
2.   Share the latest Colab Notebook with Collaborators
3.   Share the updated UTXO folder with Collaborators

Up to now, 

*   Bitcoin (btc) is updated until 2021-02-10.
*   litecoin (ltc) is updated until 2020-12-31.
*   dogecoin (dgc) is updated until 2020-12-31.
*   dash (dash) is updated until 2021-02-20.
*   zcash (zec) is updated until 2020-12-31.
*   bitcoin_cash (bch) is updated until 2020-12-31.









# Preliminaries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import decimal
from datetime import datetime, date, timedelta, timezone

Please change the input listed in this code box.

In [ ]:
currency = "litecoin" #the name of the altcoin
currency_brief = "ltc"
last_update = date(2022,5,20)
start = last_update + timedelta(days=1)
end = date(2022,5,31)

PROJECT_ID = 'sonic-glazing-354409'

In [ ]:
#Connect to Google Cloud
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
#Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')
print('Authenticated')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Authenticated


In [ ]:
#Connect to Google BigQuery
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID, location='US')
dataset_ref = client.dataset(currency+'_UTXO', project=PROJECT_ID)
dataset = client.get_dataset(dataset_ref)
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset
for table in tables:  
  print(table.table_id)

litecoin_UTXO


# Query the latest data in BigQuery

## Creating a Table for variables of interest

In [ ]:
#Create joint_all

table_id_inputs = PROJECT_ID + "." + currency + "_UTXO.joint_all"
job_config = bigquery.QueryJobConfig(destination=table_id_inputs)
job_config.write_disposition = "WRITE_TRUNCATE"

sql = """
  SELECT
    (outputs.value/POW(10,8)) AS UTXO,  
    outputs.block_timestamp,
    inputs.block_timestamp AS spent_block_timestamp,
    #FORMAT_TIMESTAMP("%Y-%m-%d", block_timestamp) AS block_date,
    #FORMAT_TIMESTAMP("%Y-%m-%d", spent_block_timestamp) AS spent_block_date,
  FROM 
    `bigquery-public-data.crypto_""" + currency + """.outputs` AS outputs
  LEFT JOIN 
    `bigquery-public-data.crypto_""" + currency + """.inputs` AS inputs
  ON outputs.transaction_hash=inputs.spent_transaction_hash  
  AND outputs.index = inputs.spent_output_index
"""

# Start the query, passing in the extra configuration.
query_job_inputs = client.query(sql, job_config=job_config)  # Make an API request.
query_job_inputs.result()  # Wait for the job to complete.

print("Query results loaded to the table {}".format(table_id_inputs))

Query results loaded to the table sonic-glazing-354409.litecoin_UTXO.joint_all


## Create partitioned tables

In [ ]:
#Partition Table by born date for data after 2012

job_config = bigquery.QueryJobConfig()
sql = """
  CREATE OR REPLACE TABLE
    `""" + PROJECT_ID + "." + currency + """_UTXO.joint_all_partitionedbyborn14`
  PARTITION BY
    DATE(block_timestamp) 
    OPTIONS(partition_expiration_days= 5000, 
    require_partition_filter=true) AS

  SELECT
    *
  FROM
    `""" + PROJECT_ID + "." + currency + """_UTXO.joint_all`
  WHERE
    block_timestamp > TIMESTAMP('2014-01-01 00:00:00+00')
"""

# Start the query, passing in the extra configuration.
query_job_inputs = client.query(sql, job_config=job_config)  # Make an API request.
query_job_inputs.result()  # Wait for the job to complete.

print("Query results loaded to the table")

Query results loaded to the table


In [ ]:
#Partition by death date for data after 2012

job_config = bigquery.QueryJobConfig()

sql = """
  CREATE OR REPLACE TABLE
    `sonic-glazing-354409.""" + currency + """_UTXO.joint_all_partitionedbydeath14`
  PARTITION BY
    DATE(spent_block_timestamp) 
  OPTIONS(partition_expiration_days= 5000, 
    require_partition_filter=true) AS
  SELECT
    *
  FROM
    `sonic-glazing-354409.""" + currency + """_UTXO.joint_all`
  WHERE
    (spent_block_timestamp > TIMESTAMP('2014-01-01 00:00:00+00')
    OR 
    spent_block_timestamp IS NULL)
"""

# Start the query, passing in the extra configuration.
query_job_inputs = client.query(sql, job_config=job_config)  # Make an API request.
query_job_inputs.result()  # Wait for the job to complete.

print("Query results loaded to the table")

Query results loaded to the table


# Process the Data

## Define Functions

In [ ]:
def cal(x):
    t=np.sign(x-0.999)+np.sign(x-29.999)+np.sign(x-90.999)+np.sign(x-181.999)+np.sign(x-364.999)+np.sign(x-365*2+0.001)+np.sign(x-365*3+0.001)+np.sign(x-365*4+0.001)+np.sign(x-365*5+0.001)+np.sign(x-365*10+0.001)+1
    return t

def Task1_born(data):
    newborn = data['UTXO'].sum()
    return(newborn)

#Partitioning By Death Date
def Task1_dead(data):
    dead = data['UTXO'].sum()
    return(dead)

def Task2(data):
    #data['Life_Length'] = data['spent_block_timestamp']- data['block_timestamp']
    #data['Life_Length'] = data['Life_Length'].map(lambda x:x.days).apply(float)
    sumUTXO = data['UTXO'].sum()
    sumLength = (data['UTXO']*data['Life_Length']).sum()
    if sumUTXO == 0:
        WALE = 0.0
    else:
        WALE = sumLength/sumUTXO
    return(WALE)
def Task3(data):
    data['Life_Length'] = data['spent_block_timestamp']- data['block_timestamp']
    data['Life_Length'] = data['Life_Length'].map(lambda x:x.days).apply(float)
    data['categorical'] = cal(data['Life_Length'])
    categories = [-9, -7, -5, -3, -1, 1, 3, 5, 7, 9, 11]
    result=pd.DataFrame(np.zeros((1, 11)), columns=categories)
    for i in categories:  
        result.loc[:,i] = data[data['categorical']==i]['UTXO'].sum()  
    return result

def Task4(data, date):  
    categories = [-9, -7, -5, -3, -1, 1, 3, 5, 7, 9, 11]
    result=pd.DataFrame(np.zeros((1, 11)), columns=categories)
    if len(data)!= 0:
      data['Age'] = data['block_timestamp'].apply(lambda x: (date-x).days)
      data['categorical'] = cal(data['Age'])
      for i in categories: 
        result.loc[:,i] = data[data['categorical']==i]['UTXO'].sum()
    return result

In [ ]:
def STXOprogram(start, end):
  duration=pd.date_range(start=start, end=end)
  days = np.size(duration)
  categories = [-9, -7, -5, -3, -1, 1, 3, 5, 7, 9, 11]
  Result=pd.DataFrame(np.zeros((days, 11)), columns=categories)
  Result['date'] = duration

  for i in range(0, days):
    try:
      start_date = start + timedelta(days=i)
      end_date = start_date + timedelta(days=1)
    
  #Partitioning by Dead Date
      query2 = """
          SELECT 
            *
          FROM 
             `""" + PROJECT_ID + "." + currency + """_UTXO.joint_all_partitionedbydeath14`
          WHERE
            spent_block_timestamp >= TIMESTAMP('""" + str(start_date) + """ 00:00:00+00')
          AND 
            spent_block_timestamp < TIMESTAMP('""" + str(end_date) + """ 00:00:00+00')"""
      query_job2 = client.query(query2)
    # Make an API request  to run the query and return a pandas DataFrame
      data2 = query_job2.to_dataframe()  
    
    #Work on Task3
      Result.iloc[i,0:11]=list(Task3(data2).iloc[0])
    
  #Partitioning by Born Date
      query1 = """
          SELECT 
            CAST(UTXO AS FLOAT64) AS UTXO,
            CAST(block_timestamp AS STRING) AS block_timestamp,
            CAST(spent_block_timestamp AS STRING) AS spent_block_timestamp
          FROM 
            `nomadic-pipe-295915.bitcoin_UTXO.joint_all_partitionedbyborn14`
          WHERE
            block_timestamp >= TIMESTAMP('""" + str(start_date) + """ 00:00:00+00')
          AND 
            block_timestamp < TIMESTAMP('""" + str(end_date) + """ 00:00:00+00')"""
      query_job1 = client.query(query1)
    # Make an API request  to run the query and return a pandas DataFrame
      data1 = query_job1.to_dataframe()
    
    #Work on Task 1 and Task 2
      Result.loc[i,'newborn'] = Task1_born(data1)
      Result.loc[i,'dead'] = Task1_dead(data2)
      Result.loc[i,'WALE'] = Task2(data2)
      Result.columns = ['-9', '-7', '-5', '-3', '-1', '1', '3', '5', '7', '9', '11', 'date', 'newborn', 'dead', 'WALE']
    except:
      print(start_date)
  return Result

In [ ]:
def UTXOprogram(start, end):
  duration=pd.date_range(start=start, end=end)
  days = np.size(duration)
  categories = [-9, -7, -5, -3, -1, 1, 3, 5, 7, 9, 11]
  Dist_Alive=pd.DataFrame(np.zeros((days, 11)), columns=categories)
  Dist_Alive['date'] = duration
  start_date=start+timedelta(days=1) 
  end_date =end+timedelta(days=1) 
  # note the trick below, we only keep data whose block_timestamp<end_date, and spent_block_timestamp>start_date
  # must be from joint_all
  query = """
      SELECT 
          CAST(UTXO AS FLOAT64) AS UTXO,
          CAST(block_timestamp AS STRING) AS block_timestamp,
          CAST(spent_block_timestamp AS STRING) AS spent_block_timestamp
      FROM 
         `""" + PROJECT_ID + "." + currency + """_UTXO.joint_all`
      WHERE
        block_timestamp < TIMESTAMP('""" + str(end_date) + """ 00:00:00+00')
      AND 
        (spent_block_timestamp >= TIMESTAMP('""" + str(start_date) + """ 00:00:00+00')
        OR
        spent_block_timestamp IS NULL)
     """
  query_job = client.query(query)

# Make an API request  to run the query and return a pandas DataFrame
  data = query_job.to_dataframe()
  data['block_timestamp'] = pd.to_datetime(data['block_timestamp'], format='%Y-%m-%d')
  data['spent_block_timestamp'] = pd.to_datetime(data['spent_block_timestamp'].fillna(pd.NaT), format='%Y-%m-%d')
  for j in range(0, days):
    working_date = pd.to_datetime(start_date + timedelta(days=j), utc=True)   
    working_data = data.loc[((data.block_timestamp<working_date) & ((pd.isna(data.spent_block_timestamp) | (data.spent_block_timestamp>=working_date))))].copy()
    Dist_Alive.iloc[j,0:11] = list(Task4(working_data, working_date).iloc[0])

  return Dist_Alive

In [ ]:
def UTXOprogram1(start, end):
  duration=pd.date_range(start=start, end=end)
  days = np.size(duration)
  categories = [-9, -7, -5, -3, -1, 1, 3, 5, 7, 9, 11]
  Dist_Alive=pd.DataFrame(np.zeros((days, 11)), columns=categories)
  Dist_Alive['date'] = duration
  start_date=start+timedelta(days=1) 
  end_date =end+timedelta(days=1) 
  # note the trick below, we only keep data whose block_timestamp<end_date, and spent_block_timestamp>start_date
  #must be from joint_all
  query = """
      SELECT 
        *
      FROM 
        `""" + PROJECT_ID + "." + currency + """_UTXO.joint_all`
      WHERE
        block_timestamp < TIMESTAMP('""" + str(end_date) + """ 00:00:00+00')
      AND 
        spent_block_timestamp >= TIMESTAMP('""" + str(start_date) + """ 00:00:00+00')
     """
  query_job = client.query(query)

# Make an API request  to run the query and return a pandas DataFrame
  data = query_job.to_dataframe()
  data['block_timestamp'] = pd.to_datetime(data['block_timestamp'], format='%Y-%m-%d')
  data['spent_block_timestamp'] = pd.to_datetime(data['spent_block_timestamp'].fillna(pd.NaT), format='%Y-%m-%d')
  for j in range(0, days):
    working_date = pd.to_datetime(start_date + timedelta(days=j), utc=True)   
    working_data = data.loc[((data.block_timestamp<working_date) & ((pd.isna(data.spent_block_timestamp) | (data.spent_block_timestamp>=working_date))))].copy()
    Dist_Alive.iloc[j,0:11] = list(Task4(working_data, working_date).iloc[0])

  return Dist_Alive

In [ ]:
def UTXOprogram2(start, end):
  duration=pd.date_range(start=start, end=end)
  days = np.size(duration)
  categories = [-9, -7, -5, -3, -1, 1, 3, 5, 7, 9, 11]
  Dist_Alive=pd.DataFrame(np.zeros((days, 11)), columns=categories)
  Dist_Alive['date'] = duration
  start_date=start+timedelta(days=1) 
  end_date =end+timedelta(days=1) 
  # note the trick below, we only keep data whose block_timestamp<end_date, and spent_block_timestamp>start_date
  #must be from joint_all
  query = """
      SELECT 
        *
      FROM 
        `""" + PROJECT_ID + "." + currency + """_UTXO.joint_all`
      WHERE
        block_timestamp < TIMESTAMP('2018-12-31 00:00:00+00')
      AND 
        spent_block_timestamp IS NULL
     """
  query_job = client.query(query)

# Make an API request  to run the query and return a pandas DataFrame
  data = query_job.to_dataframe()
  data['block_timestamp'] = pd.to_datetime(data['block_timestamp'], format='%Y-%m-%d')
  data['spent_block_timestamp'] = pd.to_datetime(data['spent_block_timestamp'], format='%Y-%m-%d')
  for j in range(0, days):
    working_date = pd.to_datetime(start_date + timedelta(days=j), utc=True)   
    working_data = data.loc[(data.block_timestamp<working_date)].copy()
    Dist_Alive.iloc[j,0:11] = list(Task4(working_data, working_date).iloc[0])

  return Dist_Alive

In [ ]:
def UTXOprogram3(start, end):
  duration=pd.date_range(start=start, end=end)
  days = np.size(duration)
  categories = [-9, -7, -5, -3, -1, 1, 3, 5, 7, 9, 11]
  Dist_Alive=pd.DataFrame(np.zeros((days, 11)), columns=categories)
  Dist_Alive['date'] = duration
  start_date=start+timedelta(days=1) 
  end_date =end+timedelta(days=1) 
  # note the trick below, we only keep data whose block_timestamp<end_date, and spent_block_timestamp>start_date
  #must be from joint_all
  query = """
      SELECT 
        *
      FROM 
        `""" + PROJECT_ID + "." + currency + """_UTXO.joint_all`
      WHERE
        block_timestamp < TIMESTAMP('""" + str(end_date) + """ 00:00:00+00')
      AND
        block_timestamp > TIMESTAMP('2018-12-31 00:00:00+00')
      AND 
        spent_block_timestamp IS NULL
     """
  query_job = client.query(query)

# Make an API request  to run the query and return a pandas DataFrame
  data = query_job.to_dataframe()
  data['block_timestamp'] = pd.to_datetime(data['block_timestamp'], format='%Y-%m-%d')
  data['spent_block_timestamp'] = pd.to_datetime(data['spent_block_timestamp'], format='%Y-%m-%d')
  for j in range(0, days):
    working_date = pd.to_datetime(start_date + timedelta(days=j), utc=True)   
    working_data = data.loc[(data.block_timestamp<working_date)].copy()
    Dist_Alive.iloc[j,0:11] = list(Task4(working_data, working_date).iloc[0])

  return Dist_Alive

## STXO Program

In [ ]:
STXOresult = STXOprogram(start, end)

oldaddress = '/content/drive/My Drive/UTXO/' + currency + "/" + currency + 'ResultSTXO' +str(last_update)+ '.csv'
oldSTXOresult = pd.read_csv(oldaddress)
oldSTXOresult = oldSTXOresult.drop(['Unnamed: 0'], axis = 1)
newSTXOresult = oldSTXOresult.append(STXOresult)
newaddress = '/content/drive/My Drive/UTXO/' +currency+ "/" + currency + 'ResultSTXO' +str(end)+ '.csv'
newSTXOresult.to_csv(newaddress)

2022-05-21
2022-05-22
2022-05-23
2022-05-24
2022-05-25
2022-05-26
2022-05-27
2022-05-28
2022-05-29
2022-05-30
2022-05-31


## UTXO Program

In [ ]:
UTXOresult = UTXOprogram(start, end)

oldaddress = '/content/drive/My Drive/UTXO/' + currency + "/" + currency + 'ResultUTXO' +str(last_update)+ '.csv'
oldUTXOresult = pd.read_csv(oldaddress)
oldUTXOresult = oldUTXOresult.drop(['Unnamed: 0'], axis = 1)
UTXOresult.columns = ['-9', '-7', '-5', '-3', '-1', '1', '3', '5', '7', '9', '11', 'date']
UTXOresult = UTXOresult[['date', '-9', '-7', '-5', '-3', '-1', '1', '3', '5', '7', '9', '11']]
newUTXOresult = oldUTXOresult.append(UTXOresult)
newUTXOresult = newUTXOresult.reset_index(drop = True)
duration=pd.date_range(start= end-timedelta(days=len(newUTXOresult)-1), end=end)
newUTXOresult['date'] = duration
newaddress = '/content/drive/My Drive/UTXO/' + currency + "/" + currency + 'ResultUTXO' +str(end)+ '.csv'
newUTXOresult.to_csv(newaddress)

### For Bitcoin and Bitcoin Cash

The data of Bitcoin and Bitcoin Cash might exceed the limit of computer RAM. In this case, we query the data in several separate parts.

In [ ]:
UTXOresult1 = UTXOprogram1(start, end)
newaddress = '/content/drive/My Drive/UTXO/' + currency + "/" + currency + 'ResultUTXO1' +str(end)+ '.csv'
UTXOresult1.to_csv(newaddress)

KeyboardInterrupt: ignored

In [ ]:
UTXOresult2 = UTXOprogram2(start, end)
newaddress = '/content/drive/My Drive/UTXO/' + currency + "/" + currency + 'ResultUTXO2' +str(end)+ '.csv'
UTXOresult2.to_csv(newaddress)

In [ ]:
UTXOresult3 = UTXOprogram3(start, end)
newaddress = '/content/drive/My Drive/UTXO/' + currency + "/" + currency + 'ResultUTXO3' +str(end)+ '.csv'
UTXOresult3.to_csv(newaddress)

In [ ]:
newaddress1 = '/content/drive/My Drive/UTXO/' + currency + "/" + currency + 'ResultUTXO1' +str(end)+ '.csv'
newaddress2 = '/content/drive/My Drive/UTXO/' + currency + "/" + currency + 'ResultUTXO2' +str(end)+ '.csv'
newaddress3 = '/content/drive/My Drive/UTXO/' + currency + "/" + currency + 'ResultUTXO3' +str(end)+ '.csv'

UTXOresult1 = pd.read_csv(newaddress1).drop(['Unnamed: 0', 'date'], axis = 1)
UTXOresult2 = pd.read_csv(newaddress2).drop(['Unnamed: 0', 'date'], axis = 1)
UTXOresult3 = pd.read_csv(newaddress3).drop(['Unnamed: 0', 'date'], axis = 1)

UTXOresult = UTXOresult1 + UTXOresult2 + UTXOresult3

oldaddress = '/content/drive/My Drive/UTXO/' + currency + "/" + currency + 'ResultUTXO' +str(last_update)+ '.csv'
oldUTXOresult = pd.read_csv(oldaddress)
oldUTXOresult = oldUTXOresult.drop(['Unnamed: 0'], axis = 1)

duration=pd.date_range(start=start, end=end)
days = np.size(duration)
UTXOresult['date'] = duration
UTXOresult = UTXOresult[['date', '-9', '-7', '-5', '-3', '-1', '1', '3', '5', '7', '9', '11']]
newUTXOresult = oldUTXOresult.append(UTXOresult)
newUTXOresult = newUTXOresult.reset_index(drop = True)
duration=pd.date_range(start= end-timedelta(days=len(newUTXOresult)-1), end=end)
newUTXOresult['date'] = duration
newaddress = '/content/drive/My Drive/UTXO/' + currency + "/" + currency + 'ResultUTXO' +str(end)+ '.csv'
newUTXOresult.to_csv(newaddress)